Name: Hamidreza Yaghoubi Araghi

Student ID: 98109786

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab\ Notebooks/Deep\ Learning/Home\ Works/HW03/

/content/drive/MyDrive/Colab Notebooks/Deep Learning/Home Works/HW03


In [ ]:
cd transformers

/content/drive/MyDrive/Colab Notebooks/Deep Learning/Home Works/HW03/transformers


# Clone Transformer

In [ ]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 139197, done.
remote: Counting objects: 100% (1189/1189), done.
remote: Compressing objects: 100% (476/476), done.
remote: Total 139197 (delta 742), reused 987 (delta 614), pack-reused 138008
Receiving objects: 100% (139197/139197), 137.54 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (104207/104207), done.
Updating files: 100% (3245/3245), done.


# Installation

In [ ]:
!pip install -e transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/Deep%20Learning/Home%20Works/HW03/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.8 MB/s eta 0:00:00
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.29.0.dev0-0.editable-py3-none-any.whl size=36187 sha256=e3be590d6f4afed92061cef12c02bbfa2b8a2f62a7507e18f96d22f0ff178e55
  Stored in directory: /tmp/pip-ephem-wheel-cache-3m63gg0v/wheels/0f/ec/f9/6b971bd2784748ac2364b375267a8397240278588a2fb69489
Successfully built transformers
Looking in indexes: https://

# Download Dataset

In [ ]:
# !wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xvzf aclImdb_v1.tar.gz

--2023-05-04 10:10:09--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  24.8MB/s    in 5.0s    

2023-05-04 10:10:15 (16.0 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



# Prepare DataSet

In [ ]:
import datasets
from transformers import RobertaTokenizerFast

train_data, test_data = datasets.load_dataset('imdb', split =['train', 'test'])

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)

def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = True, truncation=True)

train_data = train_data.map(tokenization, batched = True)
test_data = test_data.map(tokenization, batched = True)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Training

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred, labels):
    preds = pred.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'f1': f1,
        'recall': recall,
        'accuracy': acc,
        'precision': precision,
    }

In [ ]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base')

for param in model.roberta.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.6.attention.ou

In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

for name, param in model.named_parameters():
    if 'adapter' in name:
      param.requires_grad = True

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(2):
  print(f"------------epoch={epoch}------------")
  total_loss = 0
  for batch in train_loader:

    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    total_loss += loss
    loss.backward()
    optim.step()
    i += 1
  print(f'END_LOSS = {total_loss / len(train_loader) * 4}')


------------epoch=1------------
END_LOSS = 0.8151084184646606
------------epoch=2------------
END_LOSS = 0.634392499923706



In [ ]:
from torch.utils.data import DataLoader
all_output = None
all_label = None

output_list = []
label_list = []

test_loader = DataLoader(test_data, batch_size=128, shuffle=True)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.eval()
model.to(device)
batch_number = 0
for batch in test_loader:
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['label'].to(device)
  outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
  if all_output is not None:
    all_output = torch.cat((all_output.cpu(), outputs['logits'].cpu())).cpu()
    all_label = torch.cat((all_label.cpu(), labels.cpu())).cpu()
    del(outputs)
  else:
    all_output = outputs[1].cpu()
    all_label = labels.cpu()

print(f'EVALUATE = {compute_metrics(all_output, all_label)}')

EVALUATE = {'f1': 0.9562445667922341, 'recall': 0.9576320371445154, 'accuracy': 0.9563078703703703, 'precision': 0.9548611111111112}
